# Database Exploration

By **Franklin Oliveira**

-----
This notebook contains some code written in order for me (Franklin) to get accquainted with the `crustacea` database. Here you'll find some basic data treatment and some insights that did show up as I started to understand all the information on file <font color='blue'>'Planilha geral Atualizda FINAL com correções.xlsx'</font>.

In [35]:
import datetime
import numpy as np
import pandas as pd

# pacotes para visualização rápida
import seaborn as sns
import matplotlib.pyplot as plt

# pacote para visualização principal
import altair as alt

# habilitando renderizador para notebook
alt.renderers.enable('notebook')

RendererRegistry.enable('notebook')

## Importing data...

In [2]:
excel = pd.ExcelFile('Planilha geral Atualizda FINAL com correções.xlsx')
sheet_name = excel.sheet_names

print('The excel file contains the following sheets:', sheet_name)
print('\nDatabase is in sheet:', sheet_name[0])

The excel file contains the following sheets: ['Banco de Dados Colecao Carcinol', 'Planilha1']

Database is in sheet: Banco de Dados Colecao Carcinol


In [5]:
# db = excel.parse(sheet_name[0], sep=';', encoding='utf-8-sig')
db = pd.read_csv('db.csv', sep=',', encoding='utf-8-sig')
print(f'The database has {db.shape[0]} rows and {db.shape[1]} columns.')

The database has 28689 rows and 132 columns.


/home/franklin/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (1,2,10,11,12,13,29,30,31,39,44,45,46,47,49,50,51,52,53,54,55,56,57,58,62,63,64,65,67,73,80,89,90,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,113,114,118,120,126,127,128) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [21]:
db.head(3)

,Catalog Number,Cataloger First Name,Cataloger Middle,Cataloger Last Name,Cataloged Date,on line,Input Type,Alt Cat Number,reserved,Object Condition,...,waterBody,Continent,Country,State,County,Latitude1,Longitude1,Datum,Latitude/Longitude Type,Lat/Long Method
0,MNRJcarcino 000001,NaN,NaN,Legado,1980/10/30,1,Doacao,NaN,NaN,Material Disponivel,...,Oceano Atlantico Sul,America do Sul,brasil,parana,NaN,25.830035 S,48.528374 W,NaN,Point,NaN
1,MNRJcarcino 000002,NaN,NaN,Legado,1980/10/31,1,Doacao,NaN,NaN,Material Disponivel,...,Oceano Atlantico Sul,America do Sul,brasil,pernambuco,NaN,NaN,NaN,NaN,Point,NaN
2,MNRJcarcino 000003,NaN,NaN,Legado,1980/10/31,1,Doacao,NaN,NaN,Material Disponivel,...,Oceano Atlantico Sul,America do Sul,brasil,rio de janeiro,Rio de Janeiro,23.00347 S,43.640745 W,NaN,Point,NaN


In [7]:
for col in db.columns:
    print('-', col)

- Catalog Number
- Cataloger First Name
- Cataloger Middle
- Cataloger Last Name
- Cataloged Date
- on line
- Input Type
- Alt Cat Number
- reserved
- Object Condition
- Count Amount
- associatedTaxa
- Females
- Males
- Ind
- Adults
- Young
- Larvae
- Egg Mass
- Measurement\n Records
- Remarks
- Kingdom
- Phylum1
- Subphylum1
- Class1
- Subclass1
- Infraclass1
- Superorder1
- Order1
- Suborder1
- Infraorder1
- Superfamily1
- Family1
- Subfamily1
- Tribe1
- Genus1
- Species1
- Species Author1
- Type Status1
- Qualifier1
- Determiner First Name1
- Determiner Middle1
- Determiner Last Name1
- Determined Date1
- Phylum2
- Subphylum2
- Class2
- Subclass2
- Infraclass2
- Superorder2
- Order2
- Suborder2
- Infraorder2
- Superfamily2
- Family2
- Subfamily2
- Tribe2
- Genus2
- Species2
- Species Author2
- Type Status2
- Qualifier2
- Determiner First Name2
- Determiner Middle2
- Determiner Last Name2
- Determined Date2
- Prep Type1
- Count 1
- Description 1
- Prepared First Name By1
- Prepared M

#### removing '\n' from column names

In [8]:
db.columns = [str(col).replace(r'\n','') for col in db.columns]

#### closer look on some columns...

In [9]:
db[['Species1', 'Species Author1', 'Type Status1', 'Qualifier1', 'Determiner First Name1',
      'Determiner Middle1', 'Determiner Last Name1', 'Determined Date1']]

,Species1,Species Author1,Type Status1,Qualifier1,Determiner First Name1,Determiner Middle1,Determiner Last Name1,Determined Date1
0,schmitti,"Burkenroad, 1936",NaN,NaN,Alceu,L,de Castro,NaN
1,schmitti,"Burkenroad, 1936",NaN,NaN,Alceu,L,de Castro,NaN
2,schmitti,"Burkenroad, 1936",NaN,NaN,Alceu,L,de Castro,NaN
3,schmitti,"Hobbs III, 1979",NaN,NaN,Maria Margarida,G,Correa,NaN
4,schmitti,"Burkenroad, 1936",NaN,NaN,Alceu,L,de Castro,NaN
...,...,...,...,...,...,...,...,...
28684,serejoae,"Rafinesque, 1815",NaN,NaN,Ayrton,R,Brito,2017/06/09
28685,serejoae,"Rafinesque, 1815",NaN,NaN,Ayrton,R,Brito,2017/06/10
28686,serejoae,"Rafinesque, 1815",NaN,NaN,Ayrton,R,Brito,2017/06/10
28687,serejoae,"Rafinesque, 1815",NaN,NaN,Ayrton,R,Brito,2017/06/10


## preparing data for charts...

Column: `Type Status1`

In [30]:
crustaceas = db.copy()

In [31]:
crustaceas['Type Status1'].value_counts()

paratipo         695
holotipo         285
alotipo           81
topotipo           9
neotipo            6
paralectotipo      6
sintipo            5
lectotipo          4
material tipo      3
tipo               2
co-tipo            1
Name: Type Status1, dtype: int64

Let's begin cleaning this data and lowering all cases.

In [32]:
crustaceas['Type Status1'] = crustaceas['Type Status1'].str.strip().str.lower()

In [33]:
crustaceas['Type Status1'].value_counts()

paratipo         695
holotipo         285
alotipo           81
topotipo           9
neotipo            6
paralectotipo      6
sintipo            5
lectotipo          4
material tipo      3
tipo               2
co-tipo            1
Name: Type Status1, dtype: int64

### Collecting data 

Column: `Cataloged Date`

In [38]:
Table_Date = crustaceas[['Cataloged Date','Class1','Kingdom']].copy()

counter=0
for row in Table_Date['Cataloged Date']:
#     print(row)
    if not str(row).find('/')==-1:
        dates_values = str(row).split("/")
        year = int(dates_values[0])
        month = int(dates_values[1])
        if (month>1) and (month<12):
            #store the year and month in a datetime datatype for later sorting
            dateRecord = datetime.datetime(year,month,1)
#         else:
#             #if the month is not valid, then use 1/1/1 as the date
#             dateRecord = dt.datetime(1,1,1)
#     else:
#         #if the date is nan, then use 1/1/1 as the date
#         dateRecord = dt.datetime(1,1,1)         
    d.append({'year':year, 'class':Table_Date.loc[counter,'Class1'],
              'kingdom':Table_Date.loc[counter,'Kingdom']})
    counter = counter+1
NewTable_Date = pd.DataFrame(d)
NewTable_Date.head()

,year,class,kingdom
0,1980,Malacostraca,Animalia
1,1980,Malacostraca,Animalia
2,1980,Malacostraca,Animalia
3,1980,Malacostraca,Animalia
4,1980,Malacostraca,Animalia
